# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_path="../output_data/weather_data.csv"
weather_data_df=pd.read_csv(weather_data_path,encoding="utf-8")
weather_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        550 non-null    object 
 1   lng         550 non-null    float64
 2   lat         550 non-null    float64
 3   Max Temp    550 non-null    float64
 4   Humidity    550 non-null    int64  
 5   Wind Speed  550 non-null    float64
 6   Cloudiness  550 non-null    int64  
 7   Country     546 non-null    object 
 8   Date        550 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 38.8+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_data_df[["lat", "lng"]]

# Fill NaN values and convert to float
humidity = weather_data_df["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(20.0, 80.0), zoom_level=1.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                max_intensity=100,
                                 dissipating=True,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature between 15 to 35.
# Wind speed less than 15 mph.
# Humidity in between 25-50
# cloudiness in between 10-35
Ideal_weather=weather_data_df.loc[(weather_data_df["Max Temp"]>15)&(weather_data_df["Max Temp"]<35)
                                  &(weather_data_df["Wind Speed"]<15)
                                  &(weather_data_df["Humidity"]>20)&(weather_data_df["Humidity"]<50)
                                  &(weather_data_df["Cloudiness"]>10)&(weather_data_df["Cloudiness"]<35)]
Ideal_weather.dropna(inplace = False) 
Ideal_weather.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 40 to 512
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        13 non-null     object 
 1   lng         13 non-null     float64
 2   lat         13 non-null     float64
 3   Max Temp    13 non-null     float64
 4   Humidity    13 non-null     int64  
 5   Wind Speed  13 non-null     float64
 6   Cloudiness  13 non-null     int64  
 7   Country     13 non-null     object 
 8   Date        13 non-null     int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 1.0+ KB


In [7]:
Ideal_weather.head()

,city,lng,lat,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
40,Kungurtug,97.5228,50.5994,21.24,34,1.63,32,RU,1624017924
94,Wattegama,81.4808,6.7989,33.51,48,2.86,28,LK,1624017940
112,Tiksi,128.8694,71.6872,15.12,49,1.15,34,RU,1624017944
154,Soverato Marina,16.5499,38.6850,25.97,40,1.56,27,IT,1624017954
187,Kruisfontein,24.7314,-34.0033,18.27,46,4.37,14,ZA,1624017964


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df=Ideal_weather[["Country","city","lng","lat"]].copy()
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 40 to 512
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  13 non-null     object 
 1   city     13 non-null     object 
 2   lng      13 non-null     float64
 3   lat      13 non-null     float64
dtypes: float64(2), object(2)
memory usage: 520.0+ bytes


In [9]:
# set up additional columns to hold information
hotel_df.loc[:,'Hotel Name'] = ""
hotel_df.loc[:,'Address'] = ""

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Find a hotel based on city_coords.
target_search = "Hotels"
radius = 5000
city_coords = hotel_df[["lng","lat"]]

# redefine params
params = {
    "location": city_coords,
    "keyword": target_search,
    "radius": radius,
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat=row["lat"]
    lng=row["lng"]
    city=row["city"]
    # get city location from df
    city_coords = f"{lat}, {lng}"
    print (city_coords)
    # add keyword to params dict
    params['location'] = city_coords

    # assemble url and make API request
   
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Address'] = results[0]['vicinity']
        print(f"Closest hotel is {results[0]['name']} for {city} .")
        print(f"Closest hotel {results[0]['name']} address is {results[0]['vicinity']} .")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


50.5994, 97.5228
Missing field/result... skipping.
6.7989, 81.4808
Missing field/result... skipping.
71.6872, 128.8694
Closest hotel is Arktika for Tiksi .
Closest hotel Arktika address is Ulitsa Polyarnoy Aviatsii, д. 4, Tiksi .
38.685, 16.5499
Closest hotel is Villa Ersilia for Soverato Marina .
Closest hotel Villa Ersilia address is Via Trento e Trieste, 140, Soverato .
-34.0033, 24.7314
Missing field/result... skipping.
57.9731, 31.6481
Missing field/result... skipping.
-16.7725, 31.5838
Missing field/result... skipping.
44.6914, 44.1744
Missing field/result... skipping.
37.15, 122.3833
Closest hotel is Jinjiang Inn for Yatou .
Closest hotel Jinjiang Inn address is Chengshan Ave Middle Section, Rongcheng, Weihai .
-13.2, 49.05
Closest hotel is Hotel National for Ambilobe .
Closest hotel Hotel National address is 6, Ambilobe .
33.0086, 70.0697
Missing field/result... skipping.
48.0056, 91.6419
Closest hotel is Steppe hotel for Khovd .
Closest hotel Steppe hotel address is magsarjav 

In [10]:
hotel_df

,Country,city,lng,lat,Hotel Name,Address
40,RU,Kungurtug,97.5228,50.5994,,
94,LK,Wattegama,81.4808,6.7989,,
112,RU,Tiksi,128.8694,71.6872,Arktika,"Ulitsa Polyarnoy Aviatsii, д. 4, Tiksi"
154,IT,Soverato Marina,16.5499,38.6850,Villa Ersilia,"Via Trento e Trieste, 140, Soverato"
187,ZA,Kruisfontein,24.7314,-34.0033,,
234,RU,Parfino,31.6481,57.9731,,
248,ZW,Mount Darwin,31.5838,-16.7725,,
262,RU,Orlovka,44.1744,44.6914,,
292,CN,Yatou,122.3833,37.1500,Jinjiang Inn,"Chengshan Ave Middle Section, Rongcheng, Weihai"
319,MG,Ambilobe,49.0500,-13.2000,Hotel National,"6, Ambilobe"


In [12]:
#drop the Nan value

hotel_df=hotel_df.replace("", np.nan, inplace=False)

hotel_df.head(8)

,Country,city,lng,lat,Hotel Name,Address
40,RU,Kungurtug,97.5228,50.5994,NaN,NaN
94,LK,Wattegama,81.4808,6.7989,NaN,NaN
112,RU,Tiksi,128.8694,71.6872,Arktika,"Ulitsa Polyarnoy Aviatsii, д. 4, Tiksi"
154,IT,Soverato Marina,16.5499,38.6850,Villa Ersilia,"Via Trento e Trieste, 140, Soverato"
187,ZA,Kruisfontein,24.7314,-34.0033,NaN,NaN
234,RU,Parfino,31.6481,57.9731,NaN,NaN
248,ZW,Mount Darwin,31.5838,-16.7725,NaN,NaN
262,RU,Orlovka,44.1744,44.6914,NaN,NaN


In [14]:
hotel_df_clean=hotel_df.dropna(inplace=False)
hotel_df_clean

,Country,city,lng,lat,Hotel Name,Address
112,RU,Tiksi,128.8694,71.6872,Arktika,"Ulitsa Polyarnoy Aviatsii, д. 4, Tiksi"
154,IT,Soverato Marina,16.5499,38.6850,Villa Ersilia,"Via Trento e Trieste, 140, Soverato"
292,CN,Yatou,122.3833,37.1500,Jinjiang Inn,"Chengshan Ave Middle Section, Rongcheng, Weihai"
319,MG,Ambilobe,49.0500,-13.2000,Hotel National,"6, Ambilobe"
462,MN,Khovd,91.6419,48.0056,Steppe hotel,"magsarjav bag, 4th street 4-2 Ховд, sum"
512,CI,Odienné,-7.5692,9.5100,HÔTEL SARAH,"Route de de Sous-Péfecture, Odienné"


In [15]:
addresses=hotel_df_clean['Address'].tolist()
addresses

['Ulitsa Polyarnoy Aviatsii, д. 4, Tiksi',
 'Via Trento e Trieste, 140, Soverato',
 'Chengshan Ave Middle Section, Rongcheng, Weihai',
 '6, Ambilobe',
 'magsarjav bag, 4th street 4-2 Ховд, sum',
 'Route de de Sous-Péfecture, Odienné']

In [16]:

# Create a map using city coordinates to set markers

# Create a marker_layer using the hotel info to fill the info box

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Hotel Adress</dt><dd>{Address}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]
marker_locations = hotel_df_clean[['lat', 'lng']]
fig = gmaps.figure(center=(20.0, 80.0), zoom_level=1.5)
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(markers)
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))